# 8-Puzzle Solver

In [32]:
from collections import deque
import time
import psutil
import os

## get_ram_usage()
Função para obter o total de memória ram usada (em MB) na execução dos algoritmos

In [33]:
def get_ram_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # MB

## rebuild_path(start, finish)
Função para reconstruir o caminho feito entre dois estados do puzzle.

Seu propósito é puramente por fins de economia de memória. Com ela, é possível guardar apenas as transições de estado entre os nós da árvore e reconstruir o caminho correto no final, ao invés de guardar todos os caminhos explorados.
### Exemplo de uso
Seja o estado inicial (1, 2, 3, 4, 5, 6, 7, 0, 8) e o estado final (1, 2, 3, 4, 5, 6, 7, 8, 0):
Nessa instância, a função retorna "Right", já que o 0 foi trocado de lugar com o número 8 (e o número 8 por sua vez estava à direita do 0 no puzzle)

In [34]:
def rebuild_path(start, finish):
    i = start.index("9")
    if i > 2:
        cpy = list(start)
        cpy[i], cpy[i - 3] = cpy[i - 3], cpy[i]
        if "".join(cpy) == finish:
            return "Up"
    if i < 6:
        cpy = list(start)
        cpy[i], cpy[i + 3] = cpy[i + 3], cpy[i]
        if "".join(cpy) == finish:
            return "Down"
    if i % 3 != 0:
        cpy = list(start)
        cpy[i], cpy[i - 1] = cpy[i - 1], cpy[i]
        if "".join(cpy) == finish:
            return "Left"
    if i % 3 != 2:
        cpy = list(start)
        cpy[i], cpy[i + 1] = cpy[i + 1], cpy[i]
        if "".join(cpy) == finish:
            return "Right"

## get_next(a, b, current, tree, ds, depth):
Função para obter o próximo estado do puzzle. Primeiro é feita a troca do 0 com o número adjacente (cima, baixo, esquerda ou direita). Depois, é verificado se o estado já foi visitado. Se não, ele é adicionado à lista/pilha (dependendo do algoritmo) e o nó é adicionado à árvore.

In [35]:
def get_next(a, b, current, tree, ds, depth):
    l = list(current)
    l[a], l[b] = l[b], l[a]
    target = int(''.join(l))
    if target not in tree:
        ds.append((depth + 1, target))
        tree[target] = current

## bfs(puzzle)
### Algoritmo de busca em largura
A função utiliza o algoritmo de busca em largura para resolver o puzzle. Mas antes disso são feitas algumas otimizações:
- O código `"".join([str(x) for x in puzzle])` transforma a tupla inicial em uma string, já que a tupla ocupava mais espaço na memória.
- Em `.replace("0", "9")` o 0 é substituído pelo 9, e logo em seguida a string é convertida em um inteiro. Essa substituição serve para evitar casos como `"012345678"`, que seriam convertidos para `12345678` e o 0 seria perdido.
Com isso, é possível transformar os 9 números da tuppla inicial em um inteiro de 9 dígitos, o que ocupa um espaço 9x menor na memória.

Após isso, o algoritmo de busca em largura é executado normalmente de maneira não recursiva.

In [36]:
def bfs(puzzle):
    puzzle = int("".join([str(x) for x in puzzle]).replace("0", "9"))
    GOAL = "123456789"
    queue = deque()
    queue.append((0, puzzle))
    visited = set()
    tree = {puzzle: -1}
    data = {
        "path_to_goal": [],
        "cost_of_path": 0,
        "nodes_expanded": 0,
        "fringe_size": 0,
        "max_fringe_size": 0,
        "search_depth": 0,
        "max_search_depth": 0,
        "running_time": time.time(),
        "max_ram_usage": 0
    }
    while queue:
        data["max_fringe_size"] = max(data["max_fringe_size"], len(queue))
        data["max_ram_usage"] = max(data["max_ram_usage"], get_ram_usage())

        depth, current = queue.popleft()
        if current in visited:
            continue
        visited.add(current)
        current = str(current)
        data["nodes_expanded"] += 1
        data["search_depth"] = max(data["search_depth"], depth)
        data["max_search_depth"] = max(data["max_search_depth"], depth)
        if current == GOAL:
            current = int(current)
            while tree[current] != -1:
                data["path_to_goal"].append(rebuild_path(tree[current], str(current)))
                current = int(tree[current])
            data["path_to_goal"].reverse()
            data["cost_of_path"] = len(data["path_to_goal"])
            data["fringe_size"] = len(queue)
            data["running_time"] = time.time() - data["running_time"]
            return data

        i = current.index('9')
        if i % 3 != 2: # right
            get_next(i, i + 1, current, tree, queue, depth)
        if i % 3 != 0: # left
            get_next(i, i - 1, current, tree, queue, depth)
        if i < 6: # down
            get_next(i, i + 3, current, tree, queue, depth)
        if i > 2: # up
            get_next(i, i - 3, current, tree, queue, depth)
    return "Sem solução"

## dfs(puzzle)
### Algoritmo de busca em profundidade
O funcionamento do algoritmo de busca em profundidade se dá de maneira exatamente igual ao algoritmo de busca em largura, com a diferença de que é utilizada uma pilha ao invés de uma fila.

In [37]:
def dfs(puzzle):
    puzzle = int("".join([str(x) for x in puzzle]).replace("0", "9"))
    GOAL = "123456789"
    stack = deque()
    stack.append((0, puzzle))
    visited = set()
    tree = {puzzle: -1}
    data = {
        "path_to_goal": [],
        "cost_of_path": 0,
        "nodes_expanded": 0,
        "fringe_size": 0,
        "max_fringe_size": 0,
        "search_depth": 0,
        "max_search_depth": 0,
        "running_time": time.time(),
        "max_ram_usage": 0
    }
    while stack:
        data["max_fringe_size"] = max(data["max_fringe_size"], len(stack))
        data["max_ram_usage"] = max(data["max_ram_usage"], get_ram_usage())

        depth, current = stack.pop()
        if current in visited:
            continue
        visited.add(current)
        current = str(current)
        data["nodes_expanded"] += 1
        data["search_depth"] = max(data["search_depth"], depth)
        data["max_search_depth"] = max(data["max_search_depth"], depth)
        if current == GOAL:
            current = int(current)
            while tree[current] != -1:
                data["path_to_goal"].append(rebuild_path(tree[current], str(current)))
                current = int(tree[current])
            data["path_to_goal"].reverse()
            data["cost_of_path"] = len(data["path_to_goal"])
            data["fringe_size"] = len(stack)
            data["running_time"] = time.time() - data["running_time"]
            return data

        i = current.index('9')
        if i % 3 != 2: # right
            get_next(i, i + 1, current, tree, stack, depth)
        if i % 3 != 0: # left
            get_next(i, i - 1, current, tree, stack, depth)
        if i > 2: # up
            get_next(i, i - 3, current, tree, stack, depth)
        if i < 6: # down
            get_next(i, i + 3, current, tree, stack, depth)
    return "Sem solução"



## idfs(puzzle)
### Algoritmo de busca em profundidade iterativa
Do mesmo modo, o algoritmo de busca em profundidade iterativa funciona como o algoritmo de busca em profundidade. Porém, ele introduz um limite de profundidade que vai sendo incrementado a cada iteração, o que faz com que ele  explore mais caminhos, assemelhando-se mais ao algoritmo de busca em largura.

In [38]:
def idfs(puzzle):
    puzzle = int("".join([str(x) for x in puzzle]).replace("0", "9"))
    GOAL = "123456789"
    limit = 0
    data = {
        "path_to_goal": [],
        "cost_of_path": 0,
        "nodes_expanded": 0,
        "fringe_size": 0,
        "max_fringe_size": 0,
        "search_depth": 0,
        "max_search_depth": 0,
        "running_time": time.time(),
        "max_ram_usage": 0
    }

    while True:
        stack = deque()
        stack.append((0, puzzle))
        visited = {puzzle: 0}
        tree = {puzzle: -1}

        while stack:
            data["max_fringe_size"] = max(data["max_fringe_size"], len(stack))
            data["max_ram_usage"] = max(data["max_ram_usage"], get_ram_usage())

            depth, current = stack.pop()
            if depth > limit:
                continue
            current_str = str(current)
            data["nodes_expanded"] += 1
            data["search_depth"] = depth
            data["max_search_depth"] = max(data["max_search_depth"], depth)
            if current_str == GOAL:
                while tree[current] != -1:
                    data["path_to_goal"].append(rebuild_path(str(tree[current]), str(current)))
                    current = int(tree[current])
                data["path_to_goal"].reverse()
                data["cost_of_path"] = len(data["path_to_goal"])
                data["fringe_size"] = len(stack)
                data["running_time"] = time.time() - data["running_time"]
                return data

            i = current_str.index('9')
            neighbors = []
            if i % 3 != 2:
                l = list(current_str)
                l[i], l[i + 1] = l[i + 1], l[i]
                neighbors.append(int(''.join(l)))
            if i % 3 != 0:
                l = list(current_str)
                l[i], l[i - 1] = l[i - 1], l[i]
                neighbors.append(int(''.join(l)))
            if i < 6:
                l = list(current_str)
                l[i], l[i + 3] = l[i + 3], l[i]
                neighbors.append(int(''.join(l)))
            if i > 2:
                l = list(current_str)
                l[i], l[i - 3] = l[i - 3], l[i]
                neighbors.append(int(''.join(l)))

            for neighbor in reversed(neighbors):
                if neighbor not in visited or visited[neighbor] > depth + 1:
                    visited[neighbor] = depth + 1
                    if depth < limit:
                        stack.append((depth + 1, neighbor))
                        tree[neighbor] = current

        limit += 1

# Exemplo de uso

In [ ]:
puzzle = (0, 8, 7, 6, 5, 4, 3, 2, 1)
print("BFS")
for key, value in bfs(puzzle).items():
    print(f"{key}: {value}")
print("DFS")
for key, value in dfs(puzzle).items():
    print(f"{key}: {value}")
print("IDFS")
for key, value in idfs(puzzle).items():
    print(f"{key}: {value}")

## Análise
É possível observar que, dentre as 3 opções de algoritmos, o DFS obteve a solução mais longa, o IDFS obteve a menor solução, mas demorou em média 13x mais que os outros algoritmos, e, por fim, o BFS apresentou um equilíbrio entre velocidade e solução, sendo a melhor opção no geral.

In [ ]:
puzzle = (1,2,3,4,5,6,7,0,8)
print("BFS")
for key, value in bfs(puzzle).items():
    print(f"{key}: {value}")
print("DFS")
for key, value in dfs(puzzle).items():
    print(f"{key}: {value}")
print("IDFS")
for key, value in idfs(puzzle).items():
    print(f"{key}: {value}")

## Análise
Nesse caso, os algoritmos BFS e IDFS chegam na solução de forma quase que instantânea, já que exploram mais opções de maneira imediata. Diferente do algoritmo DFS, que checa primeiro todas as opções que começam com "Up" para depois testar a próxima direção e chegar na solução. É possível otimizar o DFS para essa instância mudando a ordem em que os nós da árvore são checados(atualmente Down > Up > Left > Right), mas isso pioraria o desempenho de outras instâncias, o que no final não traria beneficio algum.

Abaixo uma instância onde o DFS possui o mesmo desempenho dos outros algoritmos

In [ ]:
puzzle = (1,2,3,4,5,0,7,8,6)
print("BFS")
for key, value in bfs(puzzle).items():
    print(f"{key}: {value}")
print("DFS")
for key, value in dfs(puzzle).items():
    print(f"{key}: {value}")
print("IDFS")
for key, value in idfs(puzzle).items():
    print(f"{key}: {value}")